In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

DATA_PATH = "/content/drive/MyDrive/Datathon_leeya/data/"
SEED = 42

In [3]:
teams_train = pd.read_csv(f"{DATA_PATH}TEST3_train_0114.csv")
teams_test = pd.read_csv(f"{DATA_PATH}TEST3_test_0114.csv")
teams_train_target = pd.read_csv(f"{DATA_PATH}teams_train_target.csv")
teams_test_target = pd.read_csv(f"{DATA_PATH}teams_test_target.csv")

In [4]:
teams_train.shape, teams_train_target.shape, teams_test.shape, teams_test_target.shape

((9913, 491), (9913, 3), (2324, 491), (2324, 3))

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
teams_train.head(10)

,gameid,patch,side,league,teamname,opp_teamname,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,year,month,day,hour,hour_sin,hour_cos,game_per_day,time_period,minute,recent10_result,recent10_result.1,recent10_result.2,recent10_result.3,opp_recent10_result,opp_recent10_result.1,opp_recent10_result.2,opp_recent10_result.3,recent10_gamelength,recent10_gamelength.1,recent10_gamelength.2,recent10_gamelength.3,opp_recent10_gamelength,opp_recent10_gamelength.1,opp_recent10_gamelength.2,opp_recent10_gamelength.3,recent10_kills,recent10_kills.1,recent10_kills.2,recent10_kills.3,opp_recent10_kills,opp_recent10_kills.1,opp_recent10_kills.2,opp_recent10_kills.3,recent10_deaths,recent10_deaths.1,recent10_deaths.2,recent10_deaths.3,opp_recent10_deaths,opp_recent10_deaths.1,opp_recent10_deaths.2,opp_recent10_deaths.3,recent10_assists,recent10_assists.1,recent10_assists.2,recent10_assists.3,opp_recent10_assists,opp_recent10_assists.1,opp_recent10_assists.2,opp_recent10_assists.3,recent10_firstblood,recent10_firstblood.1,recent10_firstblood.2,recent10_firstblood.3,opp_recent10_firstblood,opp_recent10_firstblood.1,opp_recent10_firstblood.2,opp_recent10_firstblood.3,recent10_team kpm,recent10_team kpm.1,recent10_team kpm.2,recent10_team kpm.3,opp_recent10_team kpm,opp_recent10_team kpm.1,opp_recent10_team kpm.2,opp_recent10_team kpm.3,recent10_ckpm,recent10_ckpm.1,recent10_ckpm.2,recent10_ckpm.3,opp_recent10_ckpm,opp_recent10_ckpm.1,opp_recent10_ckpm.2,opp_recent10_ckpm.3,recent10_firstdragon,recent10_firstdragon.1,recent10_firstdragon.2,recent10_firstdragon.3,opp_recent10_firstdragon,opp_recent10_firstdragon.1,opp_recent10_firstdragon.2,opp_recent10_firstdragon.3,recent10_firstherald,recent10_firstherald.1,recent10_firstherald.2,recent10_firstherald.3,opp_recent10_firstherald,opp_recent10_firstherald.1,opp_recent10_firstherald.2,opp_recent10_firstherald.3,recent10_void_grubs,recent10_void_grubs.1,recent10_void_grubs.2,recent10_void_grubs.3,opp_recent10_void_grubs,opp_recent10_void_grubs.1,opp_recent10_void_grubs.2,opp_recent10_void_grubs.3,recent10_firstbaron,recent10_firstbaron.1,recent10_firstbaron.2,recent10_firstbaron.3,opp_recent10_firstbaron,opp_recent10_firstbaron.1,opp_recent10_firstbaron.2,opp_recent10_firstbaron.3,recent10_firsttower,recent10_firsttower.1,recent10_firsttower.2,recent10_firsttower.3,opp_recent10_firsttower,opp_recent10_firsttower.1,opp_recent10_firsttower.2,opp_recent10_firsttower.3,recent10_towers,recent10_towers.1,recent10_towers.2,recent10_towers.3,opp_recent10_towers,opp_recent10_towers.1,opp_recent10_towers.2,opp_recent10_towers.3,recent10_firstmidtower,recent10_firstmidtower.1,recent10_firstmidtower.2,recent10_firstmidtower.3,opp_recent10_firstmidtower,opp_recent10_firstmidtower.1,opp_recent10_firstmidtower.2,opp_recent10_firstmidtower.3,recent10_firsttothreetowers,recent10_firsttothreetowers.1,recent10_firsttothreetowers.2,recent10_firsttothreetowers.3,opp_recent10_firsttothreetowers,opp_recent10_firsttothreetowers.1,opp_recent10_firsttothreetowers.2,opp_recent10_firsttothreetowers.3,recent10_turretplates,recent10_turretplates.1,recent10_turretplates.2,recent10_turretplates.3,opp_recent10_turretplates,opp_recent10_turretplates.1,opp_recent10_turretplates.2,opp_recent10_turretplates.3,recent10_inhibitors,recent10_inhibitors.1,recent10_inhibitors.2,recent10_inhibitors.3,opp_recent10_inhibitors,opp_recent10_inhibitors.1,opp_recent10_inhibitors.2,opp_recent10_inhibitors.3,recent10_damagetochampions,recent10_damagetochampions.1,recent10_damagetochampions.2,recent10_damagetochampions.3,opp_recent10_damagetochampions,opp_recent10_damagetochampions.1,opp_recent10_damagetochampions.2,opp_recent10_damagetochampions.3,recent10_dpm,recent10_dpm.1,recent10_dpm.2,recent10_dpm.3,opp_recent10_dpm,opp_recent10_dpm.1,opp_recent10_dpm.2,opp_recent10_dpm.3,recent10_damagetakenperminute,recent10_damagetakenperminute.1,recent10_damagetakenperminute.2,recent10_damagetakenperminute.3,opp_recent10_damagetakenperminute,opp_recent1

In [7]:
teams_train.select_dtypes("object").columns, teams_test.select_dtypes("object").columns

(Index(['gameid', 'side', 'league', 'teamname', 'opp_teamname', 'ban1', 'ban2',
        'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5'],
       dtype='object'),
 Index(['gameid', 'side', 'league', 'teamname', 'opp_teamname', 'ban1', 'ban2',
        'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5'],
       dtype='object'))

In [8]:
teams_train["side"] = teams_train["side"].map({"Blue": 0, "Red": 1}) # 진영 인코딩
teams_test["side"] = teams_test["side"].map({"Blue": 0, "Red": 1})

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def preprocess(train_ft, test_ft):
    champion_columns_teams = ['ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5'] # 챔피언 레이블인코딩

    champions = pd.concat([
        train_ft[champion_columns_teams],
        test_ft[champion_columns_teams],
    ]).stack().unique()

    champions_df = pd.DataFrame({'champion': champions})
    champions_df = champions_df.dropna().reset_index(drop=True)

    le = LabelEncoder()
    champions_df['champion_encoded'] = le.fit_transform(champions_df['champion'])

    for col in champion_columns_teams:
        train_ft[col] = le.transform(train_ft[col])
        test_ft[col] = le.transform(test_ft[col])

    encoder = OneHotEncoder() # 리그 원핫인코딩
    league_encoded = encoder.fit_transform(train_ft[["league"]]).toarray()
    league_cols = [f"league_{col}" for col in encoder.categories_[0]]
    train_ft = pd.concat(
        [train_ft, pd.DataFrame(league_encoded, columns=league_cols)], axis=1
    )
    train_ft.drop("league", axis=1, inplace=True)

    league_encoded = encoder.transform(test_ft[["league"]]).toarray()
    test_ft = pd.concat(
        [test_ft, pd.DataFrame(league_encoded, columns=league_cols)], axis=1
    )
    test_ft.drop("league", axis=1, inplace=True)

    le_team = LabelEncoder()
    all_team_names = pd.concat(
        [
            train_ft["teamname"],
            test_ft["teamname"],
            train_ft["opp_teamname"],
            test_ft["opp_teamname"],
        ]
    )
    le_team.fit(all_team_names)

    train_ft["teamname"] = le_team.transform(train_ft["teamname"])
    train_ft["opp_teamname"] = le_team.transform(train_ft["opp_teamname"])

    test_ft["teamname"] = le_team.transform(test_ft["teamname"])
    test_ft["opp_teamname"] = le_team.transform(test_ft["opp_teamname"])

    return train_ft, test_ft

In [10]:
teams_train, teams_test = preprocess(teams_train, teams_test)

print(teams_train.shape, teams_test.shape)
print(teams_train.select_dtypes("object").columns)
print(teams_test.select_dtypes("object").columns)


(9913, 498) (2324, 498)
Index(['gameid'], dtype='object')
Index(['gameid'], dtype='object')


In [11]:
cutoff_patch = teams_train["patch"].quantile(0.8)
train_games = teams_train[teams_train["patch"] < cutoff_patch]["gameid"].unique()
valid_games = teams_train[teams_train["patch"] >= cutoff_patch]["gameid"].unique()

train_x = teams_train[teams_train["gameid"].isin(train_games)]
valid_x = teams_train[teams_train["gameid"].isin(valid_games)]

train_y = teams_train_target[teams_train_target["gameid"].isin(train_games)]["result"]
valid_y = teams_train_target[teams_train_target["gameid"].isin(valid_games)]["result"]

train_x.drop(columns=["gameid"], inplace=True)
valid_x.drop(columns=["gameid"], inplace=True)

train_x.shape, valid_x.shape

<ipython-input-11-ac74f521d9ea>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x.drop(columns=["gameid"], inplace=True)
<ipython-input-11-ac74f521d9ea>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_x.drop(columns=["gameid"], inplace=True)


((7799, 497), (2114, 497))

In [12]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [13]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [14]:
# from sklearn.metrics import classification_report

# # 모델 리스트
# models = [
#     LogisticRegression(random_state=SEED),
#     LGBMClassifier(random_state=SEED, n_jobs=-1),
#     RandomForestClassifier(random_state=SEED, n_jobs=-1),
#     HistGradientBoostingClassifier(random_state=SEED),
#     AdaBoostClassifier(random_state=SEED),
#     SVC(random_state=SEED, probability=True),
#     XGBClassifier(random_state=SEED, n_jobs=-1),
#     CatBoostClassifier(random_state=SEED, verbose=0),
# ]

# # 결과 출력
# for model in models:
#     model_name = model.__class__.__name__
#     model.fit(train_x, train_y)
#     valid_pred = model.predict(valid_x)

#     print(f"\n=== {model_name} ===")
#     print(classification_report(valid_y, valid_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



=== LogisticRegression ===
              precision    recall  f1-score   support

           0       0.49      0.48      0.49      1060
           1       0.49      0.50      0.50      1054

    accuracy                           0.49      2114
   macro avg       0.49      0.49      0.49      2114
weighted avg       0.49      0.49      0.49      2114

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3903, number of negative: 3896
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.240264 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 82673
[LightGBM] [Info] Number of data points in the train set: 7799, number of used features: 497
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500449 -> initscore=0.001795
[LightGBM] [Info] Start training from score 0.001795


KeyboardInterrupt: 

In [15]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00


In [16]:
import optuna
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import numpy as np

In [17]:
# 공통 평가 함수 (Optuna에서 사용)
def evaluate_model(model, X, y):
    """Cross-validation을 사용해 모델 성능을 평가하는 함수."""
    scores = cross_val_score(model, X, y, cv=3, scoring='accuracy', n_jobs=-1)
    return np.mean(scores)

# 각 모델의 Optuna 튜닝
def optimize_lgbm(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
    }
    model = LGBMClassifier(random_state=SEED, **params)
    return evaluate_model(model, train_x, train_y)

def optimize_histgb(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_iter': trial.suggest_int('max_iter', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 50),
        'l2_regularization': trial.suggest_loguniform('l2_regularization', 1e-6, 1.0),
    }
    model = HistGradientBoostingClassifier(random_state=SEED, **params)
    return evaluate_model(model, train_x, train_y)

def optimize_adaboost(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
    }
    model = AdaBoostClassifier(random_state=SEED, **params)
    return evaluate_model(model, train_x, train_y)

def optimize_xgb(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
    }
    model = XGBClassifier(random_state=SEED, use_label_encoder=False, eval_metric='logloss', **params)
    return evaluate_model(model, train_x, train_y)

# def optimize_catboost(trial):
#     params = {
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
#         'depth': trial.suggest_int('depth', 3, 12),
#         'iterations': trial.suggest_int('iterations', 50, 300),
#         'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1, 10),
#     }
#     model = CatBoostClassifier(random_state=SEED, verbose=0, **params)
#     return evaluate_model(model, train_x, train_y)

In [21]:
def optimize_catboost(trial):
    # 하이퍼파라미터 범위 정의
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'iterations': trial.suggest_int('iterations', 50, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
    }

    # 모델 생성 및 학습
    model = CatBoostClassifier(**params, random_seed=42, verbose=0)
    model.fit(train_x, train_y, eval_set=(valid_x, valid_y), early_stopping_rounds=20, verbose=0)

    # 검증 데이터에 대한 점수 계산
    valid_pred = model.predict(valid_x)
    accuracy = accuracy_score(valid_y, valid_pred)

    # 조기 중단 조건
    if accuracy < 0.75:  # 성능 기준 설정 (예: 0.7 미만이면 중단)
        raise optuna.exceptions.TrialPruned()

    return accuracy


In [22]:
for model_name, optimize_func in [
    # ("LightGBM", optimize_lgbm),
    # ("HistGradientBoosting", optimize_histgb),
    # ("AdaBoost", optimize_adaboost),
    # ("XGBoost", optimize_xgb),
    ("CatBoost", optimize_catboost),
]:
    print(f"=== Optimizing {model_name} ===")
    study = optuna.create_study(direction='maximize')
    study.optimize(optimize_catboost, n_trials=50)


    print(f"Best Parameters for {model_name}: {study.best_params}")
    print(f"Best Accuracy for {model_name}: {study.best_value}")


[I 2025-01-15 00:51:06,722] A new study created in memory with name: no-name-04704711-36fd-42e4-8691-a078d1e8c3a8


=== Optimizing CatBoost ===


[I 2025-01-15 00:51:48,958] Trial 0 pruned. 
[I 2025-01-15 00:53:19,921] Trial 1 finished with value: 0.761116367076632 and parameters: {'learning_rate': 0.20409727745863734, 'depth': 9, 'iterations': 284, 'l2_leaf_reg': 9.066797942809632}. Best is trial 1 with value: 0.761116367076632.
[I 2025-01-15 00:53:46,769] Trial 2 pruned. 
[I 2025-01-15 00:55:01,328] Trial 3 pruned. 
[I 2025-01-15 00:57:10,567] Trial 4 pruned. 
[I 2025-01-15 00:57:24,231] Trial 5 pruned. 
[I 2025-01-15 00:58:19,474] Trial 6 pruned. 
[I 2025-01-15 00:58:32,097] Trial 7 pruned. 
[I 2025-01-15 00:58:42,537] Trial 8 pruned. 
[I 2025-01-15 00:59:15,944] Trial 9 pruned. 
[I 2025-01-15 01:01:17,577] Trial 10 pruned. 
[I 2025-01-15 01:01:31,109] Trial 11 pruned. 
[I 2025-01-15 01:02:22,308] Trial 12 finished with value: 0.761116367076632 and parameters: {'learning_rate': 0.12378404062970357, 'depth': 8, 'iterations': 297, 'l2_leaf_reg': 7.297794194887743}. Best is trial 1 with value: 0.761116367076632.
[I 2025-01-15 01

Best Parameters for CatBoost: {'learning_rate': 0.15436057825957208, 'depth': 6, 'iterations': 259, 'l2_leaf_reg': 6.780878410461257}
Best Accuracy for CatBoost: 0.7630085146641438
